In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# path to dataset...
anime = pd.read_csv("../anime.csv")
rating = pd.read_csv("../rating.csv")

In [25]:
# dropping the duplicates and duplicate values
anime.dropna(axis=0, inplace=True)
rating.drop_duplicates(keep='first',inplace=True)

In [26]:
# merging the tables and renaming subsequently
fulldata = pd.merge(anime,rating,on="anime_id")
fulldata = fulldata.rename(columns={"rating_x": "overall_rating"})
fulldata = fulldata.rename(columns={"rating_y": "user_rating"})

In [27]:
# pivoting the table in order to ease the life to create the sparse matrix
data_pivot = fulldata.pivot_table(index="name",columns="user_id",values="user_rating").fillna(0)

# I tried to create the plot bar out of pivot table but for some reason its taking too long..

# data_pivot.head(5)
# data_pivot.T.plot(kind='bar', ylabel='Anme Name')
# data_pivot.columns = data_pivot.columns.astype(str)
# data_pivot.plot.bar()
# plt.show()
# df = pd.pivot_table(fulldata, values = 'user_rating', index = 'name', columns = 'user_id')
# df_temp = df.head(100)
# df_temp.columns = df_temp.columns.astype(str)
# df_temp.plot.bar()
# df.plot(kind='bar')
# plt.show()


In [28]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# converting to compressed sparse matrix (CSR)
data_matrix = csr_matrix(data_pivot.values)

# calculating nearest neighbor using cosine similarity
model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(data_matrix)

# random anime title
query_no = np.random.choice(data_pivot.shape[0]) 
distances, indices = model_knn.kneighbors(data_pivot.iloc[query_no,:].values.reshape(1, -1), n_neighbors = 9)

In [29]:
no = []
name = []
distance = []
rating = []

# looping over until the number of neighbor of nearest selected anime
for i in range(0, len(distances.flatten())):
    if i == 0:
        print(f"Recommendations for {data_pivot.index[query_no]} viewers :\n") # anime itself that is selected for recommendation
    else:
        no.append(i)
        name.append(data_pivot.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])
        rating.append(*anime[anime["name"]==data_pivot.index[indices.flatten()[i]]]["rating"].values)

dic = {"No" : no, "Anime Name" : name, "Rating" : rating}
recommendation = pd.DataFrame(data = dic)
recommendation.set_index("No", inplace = True)
recommendation

Recommendations for Super Backkom viewers :



,Anime Name,Rating
No,,
1,Contact 2,5.58
2,Super Backkom,5.22
3,I Love Sky,5.67
4,I Love Picnic,5.58
5,Ultra Dino Makera,5.35
6,Unko-san: Tsuiteru Hito ni Shika Mienai Yousei,3.25
7,Organic,5.56
8,Sekishoku Elegy,6.17
